In [5]:
import torch
import torchvision
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.utils import save_image
from torchvision.datasets import MNIST
import os

if not os.path.exists('./vae_img'):
    os.mkdir('./vae_img')


def to_img(x):
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 100
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = MNIST('./data', transform=img_transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        if torch.cuda.is_available():
            eps = torch.cuda.FloatTensor(std.size()).normal_()
        else:
            eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps)
        return eps.mul(std).add_(mu)

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparametrize(mu, logvar)
        return self.decode(z), mu, logvar


model = VAE()
if torch.cuda.is_available():
    model.cuda()

reconstruction_function = nn.MSELoss(size_average=False)


def loss_function(recon_x, x, mu, logvar):
    """
    recon_x: generating images
    x: origin images
    mu: latent mean
    logvar: latent log variance
    """
    BCE = reconstruction_function(recon_x, x)  # mse loss
    # loss = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)
    # KL divergence
    return BCE + KLD


optimizer = optim.Adam(model.parameters(), lr=1e-3)

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for batch_idx, data in enumerate(dataloader):
        img, _ = data
        img = img.view(img.size(0), -1)
        img = Variable(img)
        if torch.cuda.is_available():
            img = img.cuda()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(img)
        loss = loss_function(recon_batch, img, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(img),
                len(dataloader.dataset), 100. * batch_idx / len(dataloader),
                loss.item() / len(img)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(dataloader.dataset)))
    if epoch % 10 == 0:
        save = to_img(recon_batch.cpu().data)
        save_image(save, './vae_img/image_{}.png'.format(epoch))

torch.save(model.state_dict(), './vae.pth')

/home/pritesh/anaconda3/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))
/home/pritesh/anaconda3/lib/python3.6/site-packages/torch/nn/functional.py:1386: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Train Epoch: 0 [0/60000 (0%)]	Loss: 184.943832
Train Epoch: 0 [12800/60000 (21%)]	Loss: 47.996803
Train Epoch: 0 [25600/60000 (43%)]	Loss: 43.075100
Train Epoch: 0 [38400/60000 (64%)]	Loss: 38.868206
Train Epoch: 0 [51200/60000 (85%)]	Loss: 39.957672
====> Epoch: 0 Average loss: 45.7644
Train Epoch: 1 [0/60000 (0%)]	Loss: 37.901840
Train Epoch: 1 [12800/60000 (21%)]	Loss: 36.814381
Train Epoch: 1 [25600/60000 (43%)]	Loss: 36.391891
Train Epoch: 1 [38400/60000 (64%)]	Loss: 33.807175
Train Epoch: 1 [51200/60000 (85%)]	Loss: 34.756691
====> Epoch: 1 Average loss: 35.3792
Train Epoch: 2 [0/60000 (0%)]	Loss: 34.263317
Train Epoch: 2 [12800/60000 (21%)]	Loss: 33.394459
Train Epoch: 2 [25600/60000 (43%)]	Loss: 33.739208
Train Epoch: 2 [38400/60000 (64%)]	Loss: 33.521381
Train Epoch: 2 [51200/60000 (85%)]	Loss: 32.771702
====> Epoch: 2 Average loss: 33.3482
Train Epoch: 3 [0/60000 (0%)]	Loss: 31.984898
Train Epoch: 3 [12800/60000 (21%)]	Loss: 33.016907
Train Epoch: 3 [25600/60000 (43%)]	Loss: 

Train Epoch: 28 [25600/60000 (43%)]	Loss: 28.370874
Train Epoch: 28 [38400/60000 (64%)]	Loss: 28.427521
Train Epoch: 28 [51200/60000 (85%)]	Loss: 29.198559
====> Epoch: 28 Average loss: 29.3741
Train Epoch: 29 [0/60000 (0%)]	Loss: 30.466782
Train Epoch: 29 [12800/60000 (21%)]	Loss: 29.986780
Train Epoch: 29 [25600/60000 (43%)]	Loss: 29.325943
Train Epoch: 29 [38400/60000 (64%)]	Loss: 29.651318
Train Epoch: 29 [51200/60000 (85%)]	Loss: 28.724045
====> Epoch: 29 Average loss: 29.3331
Train Epoch: 30 [0/60000 (0%)]	Loss: 29.313610
Train Epoch: 30 [12800/60000 (21%)]	Loss: 29.727142
Train Epoch: 30 [25600/60000 (43%)]	Loss: 28.211563
Train Epoch: 30 [38400/60000 (64%)]	Loss: 28.255718
Train Epoch: 30 [51200/60000 (85%)]	Loss: 28.768436
====> Epoch: 30 Average loss: 29.3049
Train Epoch: 31 [0/60000 (0%)]	Loss: 29.977085
Train Epoch: 31 [12800/60000 (21%)]	Loss: 30.143509
Train Epoch: 31 [25600/60000 (43%)]	Loss: 29.845795
Train Epoch: 31 [38400/60000 (64%)]	Loss: 28.856951
Train Epoch: 31 [

Train Epoch: 56 [25600/60000 (43%)]	Loss: 29.222012
Train Epoch: 56 [38400/60000 (64%)]	Loss: 28.388882
Train Epoch: 56 [51200/60000 (85%)]	Loss: 29.862772
====> Epoch: 56 Average loss: 28.6886
Train Epoch: 57 [0/60000 (0%)]	Loss: 28.329462
Train Epoch: 57 [12800/60000 (21%)]	Loss: 29.742271
Train Epoch: 57 [25600/60000 (43%)]	Loss: 26.686018
Train Epoch: 57 [38400/60000 (64%)]	Loss: 29.695826
Train Epoch: 57 [51200/60000 (85%)]	Loss: 30.380238
====> Epoch: 57 Average loss: 28.6458
Train Epoch: 58 [0/60000 (0%)]	Loss: 28.871819
Train Epoch: 58 [12800/60000 (21%)]	Loss: 30.349522
Train Epoch: 58 [25600/60000 (43%)]	Loss: 29.559113
Train Epoch: 58 [38400/60000 (64%)]	Loss: 28.951010
Train Epoch: 58 [51200/60000 (85%)]	Loss: 29.150713
====> Epoch: 58 Average loss: 28.6288
Train Epoch: 59 [0/60000 (0%)]	Loss: 29.107590
Train Epoch: 59 [12800/60000 (21%)]	Loss: 29.196468
Train Epoch: 59 [25600/60000 (43%)]	Loss: 28.378052
Train Epoch: 59 [38400/60000 (64%)]	Loss: 28.131409
Train Epoch: 59 [

Train Epoch: 84 [25600/60000 (43%)]	Loss: 28.153658
Train Epoch: 84 [38400/60000 (64%)]	Loss: 29.355854
Train Epoch: 84 [51200/60000 (85%)]	Loss: 29.288378
====> Epoch: 84 Average loss: 28.3084
Train Epoch: 85 [0/60000 (0%)]	Loss: 27.404453
Train Epoch: 85 [12800/60000 (21%)]	Loss: 28.414440
Train Epoch: 85 [25600/60000 (43%)]	Loss: 27.491173
Train Epoch: 85 [38400/60000 (64%)]	Loss: 28.670139
Train Epoch: 85 [51200/60000 (85%)]	Loss: 27.413662
====> Epoch: 85 Average loss: 28.2809
Train Epoch: 86 [0/60000 (0%)]	Loss: 28.397476
Train Epoch: 86 [12800/60000 (21%)]	Loss: 29.346588
Train Epoch: 86 [25600/60000 (43%)]	Loss: 28.932827
Train Epoch: 86 [38400/60000 (64%)]	Loss: 27.762327
Train Epoch: 86 [51200/60000 (85%)]	Loss: 26.758759
====> Epoch: 86 Average loss: 28.2750
Train Epoch: 87 [0/60000 (0%)]	Loss: 27.517982
Train Epoch: 87 [12800/60000 (21%)]	Loss: 27.858135
Train Epoch: 87 [25600/60000 (43%)]	Loss: 28.320992
Train Epoch: 87 [38400/60000 (64%)]	Loss: 28.709841
Train Epoch: 87 [